##

### Info

-  Notebook is about comparative analysis specifically about whole datasets and their possible alterings. 

Table of content:
- Similarity tests (adversarial cv).



### Setup

In [ ]:
import os
import gc
from tqdm.auto import tqdm
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from zillow.dataclasses.data_wrangling import WranglingColumnsConfig
from zillow.config.feature_engineering import FeaturesDtypeConversionConfig_v1
from zillow.utils.common import read_data, find_shared_cols, get_obj_features, find_all_cols
from zillow.config.config import load_config_no_wrap, create_config_from_dict, merge_configs
from zillow.config.paths import PROCESSED_DATA_DIR, RAW_DATA_DIR, INTERIM_DATA_DIR, REPORTS_DIR, ANALYSIS_RESULTS_DIR, DRIFTS_DIR
from zillow.analysis.statistics import (
    DriftDetectionTests, TwoSampHypothesisTests, 
    GroupBasedEffectSizes, TwoSampNonParamEffectSizes, TwoSampRelationshipEffectSizes, 
    TtestBasedEffectSizes, CategoricalEffectSizes
)

cfg = load_config_no_wrap('default')
cur_cfg = create_config_from_dict({
    'load_all_data': False,
    'main_train_path': INTERIM_DATA_DIR / 'cleaned_train_2016_v1.0.parquet',
    'main_test_path': INTERIM_DATA_DIR / 'cleaned_properties_2016_v1.0.parquet',
})
cfg = merge_configs(cfg, cur_cfg)

np.random.seed(cfg.RSEED)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")

sample_submission = pd.read_csv(RAW_DATA_DIR / "sample_submission.csv")
zillow_dictionary = pd.read_csv(RAW_DATA_DIR / "zillow_data_dictionary.csv")

if cfg.to_load_all_data:
    properties_2016 = read_data(path=INTERIM_DATA_DIR / "cleaned_properties_2016_v1.0.parquet", dtype='default')
    properties_2017 = read_data(path=INTERIM_DATA_DIR / "cleaned_properties_2017_v1.0.parquet", dtype='default')
    train_2016 = read_data(path=INTERIM_DATA_DIR / "cleaned_train_2016_v1.0.parquet", dtype='default')
    train_2017 = read_data(path=INTERIM_DATA_DIR / "cleaned_train_2017_v1.0.parquet", dtype='default')

train = read_data(cfg.main_train_path, dtype='default')
test = read_data(cfg.main_test_path, dtype='default')

wrangling_cols = WranglingColumnsConfig()
features_dtype_cfg = FeaturesDtypeConversionConfig_v1()


### Helpers

In [3]:
def plot_hist_dfs_comparison(dfs, features):
    ncols = 1
    nrows = sum(divmod(len(features), ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(20, nrows * 6))
    axes = np.atleast_2d(np.array(axes)).T
    colors = sns.color_palette("deep")

    for i, col in enumerate(features):
        r, c = divmod(i, ncols)
        ax = axes[r, c]

        # qq How to count non null values? How to choose only non null values without ~?
        lowest_samples = min(df[col].count() for df in dfs)

        for j, df in enumerate(dfs):
            # NOTE: uses sampling for equal comparison.
            x = df[df[col].notna()][col].sample(lowest_samples, random_state=cfg.RSEED)
            plot_histplot(column=col, x=x, bins=50, title=f'Comparison by {col}', label=f'Dataset {j+1}',
                          kde=True, color=colors[j], ylabel='Count', ax=ax)

    plt.tight_layout()
    plt.show()

# TODO can be improved.
def plot_histplot(column, x=None, data=None, bins="auto", title=None, figsize=(10, 6), kde=True, hist=True,
                 color='steelblue', stat='count', log_scale=False, ax=None, ylabel="", label=None):
    """
    Creates a histogram plot for numerical data.
    """
    if data is None and x is None:
        raise ValueError('Must specify data with column or series x. ')
    
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    
    # Data has to be used with a column. Else - using x as a series. 
    if data is not None:
        x = column

    if kde and not hist:
        sns.kdeplot(data=data, x=x, ax=ax, color=color, label=label)
    else:
        sns.histplot(data=data, x=x, bins=bins, kde=kde, 
                    element="step" if not hist else "bars",
                    color=color, stat=stat, ax=ax, label=label)

    if log_scale:
        ax.set_yscale('log')

    ax.grid(axis='y', alpha=0.3)

    if title: ax.set_title(title, fontsize=14)
    if ylabel: ax.set_ylabel(ylabel, fontsize=12)
    ax.set_xlabel(column, fontsize=12)

    if label:  # Only add legend if label is provided
        ax.legend()

    return ax

In [4]:
# Tests

def compute_dfs_ind_stats(df1, df2, features=None):
    if features is None:
        features = find_shared_dfs_cols(df1, df2)

    res = np.zeros(shape=(len(features), 4))
    fields = ['tt_stat', 'tt_pvalue', 'wh_stat', 'wh_pvalue']

    for i, feat in enumerate(features):
        tt, wh = compute_tt_wh_ind_stats(df1[feat], df2[feat])
        res[i, :] = [np.round(x, decimals=5) for x in [tt.statistic, tt.pvalue, wh.statistic, wh.pvalue]]

    res_df = pd.DataFrame(data=res.T, columns=features, index=fields)
    return res_df

def compute_tt_wh_ind_stats(x1, x2):
    x1_clean = x1.dropna()
    x2_clean = x2.dropna()
    
    if len(x1_clean) > 0 and len(x2_clean) > 0:
        tt = stats.ttest_ind(x1_clean, x2_clean)
        wh = stats.mannwhitneyu(x1_clean, x2_clean)

    else:
        raise ValueError("Insufficient data after removing NaN values")

    return [tt, wh]
    

### Statistical tests and Plotting

In [5]:
most_distinguishable_features = find_shared_cols(*[df.columns.to_list() for df in train_months_q4_sets])

num_dist_features = train[most_distinguishable_features].select_dtypes([np.number]).columns
cat_dist_features = train[most_distinguishable_features].select_dtypes([np.object_, 'category']).columns
bool_dist_features = train[most_distinguishable_features].select_dtypes([np.bool_]).columns

df1 = train_months_q4_sets[0].copy()
df2 = train_months_q4_sets[1].copy()
dfs = [df1, df2]

NameError: name 'train_months_q4_sets' is not defined

In [ ]:
stat_res_df = compute_dfs_ind_stats(df1, df2, num_dist_features)
display(stat_res_df)

,yardbuildingsqft17,finishedsquarefeet6,architecturalstyletypeid,garagetotalsqft,basementsqft,threequarterbathnbr,taxdelinquencyyear,heatingorsystemtypeid,yearbuilt,fireplacecnt,unitcnt,calculatedbathnbr,buildingqualitytypeid,lotsizesquarefeet,finishedfloor1squarefeet,longitude,fullbathcnt,landtaxvaluedollarcnt,poolsizesum,garagecarcnt,latitude,propertylandusetypeid,taxvaluedollarcnt,calculatedfinishedsquarefeet,bedroomcnt,parcelid,finishedsquarefeet15,numberofstories,yardbuildingsqft26,roomcnt,bathroomcnt,logerror,taxamount,finishedsquarefeet13,airconditioningtypeid,structuretaxvaluedollarcnt
tt_stat,0.55577,-1.37641,-0.98876,-1.37725,-0.46646,0.73412,-0.11526,-5.214600e-01,1.111820e+00,0.10065,4.749100e-01,5.727100e-01,1.394470e+00,-7.441800e-01,1.13900,-8.626600e-01,5.300100e-01,9.452800e-01,0.27773,0.39355,-2.102700e-01,-6.967200e-01,8.694500e-01,9.958000e-02,-1.106750e+00,1.361340e+00,-0.90227,-1.67213,-0.24048,2.287600e-01,7.813200e-01,4.474500e-01,7.074200e-01,-5.50000,-1.56428,5.527300e-01
tt_pvalue,0.57906,0.17796,0.33520,0.16858,0.67267,0.46308,0.90831,6.020700e-01,2.662500e-01,0.91986,6.348800e-01,5.668600e-01,1.632500e-01,4.567900e-01,0.25523,3.883600e-01,5.961200e-01,3.445500e-01,0.78198,0.69395,8.334600e-01,4.860000e-01,3.846300e-01,9.206800e-01,2.684400e-01,1.734500e-01,0.36768,0.09470,0.81685,8.190600e-01,4.346400e-01,6.545600e-01,4.793300e-01,0.03150,0.11790,5.804700e-01
wh_stat,2621.50000,101.00000,46.50000,415460.00000,1.00000,66296.50000,8096.50000,1.851631e+06,4.561326e+06,46910.00000,2.038260e+06,4.472106e+06,2.006993e+06,3.700172e+06,26188.00000,4.476846e+06,4.477178e+06,4.609239e+06,449.50000,436272.50000,4.499996e+06,4.532584e+06,4.619024e+06,4.460740e+06,4.424941e+06,4.595395e+06,6726.50000,213238.50000,3.00000,4.540015e+06,4.586086e+06,4.461759e+06,4.634562e+06,0.00000,440491.00000,4.580700e+06
wh_pvalue,0.56799,0.15652,0.50461,0.08186,0.80000,0.46338,0.24394,7.421400e-01,3.119300e-01,0.99752,8.234600e-01,4.689600e-01,2.012900e-01,5.874800e-01,0.37776,4.957000e-01,4.092700e-01,2.424700e-01,0.91578,0.95072,7.205200e-01,9.059100e-01,1.916900e-01,6.525600e-01,1.404700e-01,3.293700e-01,0.07666,0.06014,0.88889,7.817200e-01,3.762700e-01,3.723300e-01,1.278300e-01,0.34578,0.14145,3.846100e-01


In [ ]:
# NOTE: the stats were already chosen from only numeric columns.
alpha = cfg.stat_significant_magnitude / len(stat_res_df)
sign_num_cols = stat_res_df.T.query(f'(tt_pvalue <= {alpha}) & (wh_pvalue <= {alpha})').index
sign_num_cols = find_shared_cols(sign_num_cols, [c for c in most_distinguishable_features 
                                                 if c not in wrangling_cols.extreme_missing])

sign_mod_clip = find_shared_cols(sign_num_cols, wrangling_cols.moderate_clip)
sign_ext_clip = find_shared_cols(sign_num_cols, wrangling_cols.extreme_clip)

cur_dfs = [df1.copy(deep=True), df2.copy(deep=True)]
clips = [df1[sign_mod_clip].quantile(0.985), df1[sign_ext_clip].quantile(0.95)]
for df in cur_dfs:
    df[sign_mod_clip] = df[sign_mod_clip].clip(upper=clips[0], axis=1)
    df[sign_ext_clip] = df[sign_ext_clip].clip(upper=clips[1], axis=1)

try:
    plot_hist_dfs_comparison(cur_dfs, sign_num_cols)
except ValueError as e:
    print(e)

Number of rows must be a positive integer, not 0


<Figure size 2000x0 with 0 Axes>

In [ ]:
for c in bool_dist_features:
    for j, df in enumerate(dfs):
        counts = df[c].value_counts()
        try:
            ratio = np.round(counts[False] / counts[True], 3)
        except KeyError:
            ratio = float('inf')
        print(f"{c.capitalize()} F/T ratio for Dataset #{j + 1}:", ratio)
    print()


def run_bin_chi2(x1, x2, col):
    values = pd.concat([x1, x2], ignore_index=True)
    labels = ['Dataset_1']*len(x1) + ['Dataset_2']*len(x2)
    contingency = pd.crosstab(values, labels)

    chi2, p_val, _, _ = stats.chi2_contingency(contingency)
    print(f"{col}: χ² = {chi2:.4f}, p = {p_val:.2e}")

for c in cat_dist_features:
    run_bin_chi2(df1[c], df2[c], c)

Buildingclasstypeid F/T ratio for Dataset #1: 4964.0
Buildingclasstypeid F/T ratio for Dataset #2: inf

Storytypeid F/T ratio for Dataset #1: 1240.25
Storytypeid F/T ratio for Dataset #2: 1822.0

Poolcnt F/T ratio for Dataset #1: 4.041
Poolcnt F/T ratio for Dataset #2: 3.836

Decktypeid F/T ratio for Dataset #1: 126.308
Decktypeid F/T ratio for Dataset #2: 164.727

Pooltypeid10 F/T ratio for Dataset #1: 69.929
Pooltypeid10 F/T ratio for Dataset #2: 90.15

Pooltypeid7 F/T ratio for Dataset #1: 4.339
Pooltypeid7 F/T ratio for Dataset #2: 4.121

Hashottuborspa F/T ratio for Dataset #1: 38.72
Hashottuborspa F/T ratio for Dataset #2: 43.463

Taxdelinquencyflag F/T ratio for Dataset #1: 20.494
Taxdelinquencyflag F/T ratio for Dataset #2: 22.987

Pooltypeid2 F/T ratio for Dataset #1: 89.273
Pooltypeid2 F/T ratio for Dataset #2: 85.81

Typeconstructiontypeid F/T ratio for Dataset #1: 412.75
Typeconstructiontypeid F/T ratio for Dataset #2: 181.3

Fireplaceflag F/T ratio for Dataset #1: 380.923


### Data drifting detection tests

1. KS
2. PSI
3. KL divergency
4. JSD
5. Wassesterian distance

In [8]:
tests = [
    [DRIFTS_DIR / 'drift_detection_train_test.csv', DriftDetectionTests(train, test)],
    [DRIFTS_DIR / 'two_samp_hypothesis_train_test.csv', TwoSampHypothesisTests(train, test)],
    [DRIFTS_DIR / 'categorical_effect_sizes_train_test.csv', CategoricalEffectSizes(train, test)],
    [DRIFTS_DIR / 'ttest_effect_sizes_train_test.csv', TtestBasedEffectSizes(train, test)],
    [DRIFTS_DIR / 'group_effect_sizes_train_test.csv', GroupBasedEffectSizes(train, test)],
    [DRIFTS_DIR / 'relationship_effect_sizes_train_test.csv', TwoSampRelationshipEffectSizes(train, test)],
    [DRIFTS_DIR / 'non_param_effect_sizes_train_test.csv', TwoSampNonParamEffectSizes(train, test)]
]

for path, test_instance in tests:
    if os.path.exists(path):
        continue

    methods = 'all'
    if 'two_samp_hypothesis' in path.stem:
        methods = ['ttest', 'mannwhitney', 'ks', 'chi2']
    
    results = test_instance.calc_methods(methods=methods)
    results.to_csv(path)
    print(f"Saved results to {path}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

Calculated ks. 


  0%|          | 0/63 [00:00<?, ?it/s]

Calculated psi. 


...

  0%|          | 0/11 [00:00<?, ?it/s]

Calculated phi_coef. 
Saved results to D:\Coding\VSCFiles\IndependentProjects\Kaggle\zillow-2016\reports\analysis_results\drifts\relationship_effect_sizes_train_test.csv


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Conclusions

**Conclusions**:
1. Adversarial results, statistical tests and plots clearly show that properties differ, but they are anticipated to differ. 
2. Drifted distributions features: 'structuretaxvaluedollarcnt',
 'calculatedfinishedsquarefeet',
 'landtaxvaluedollarcnt',
 'finishedsquarefeet15',
 'taxamount'.
3. Differently distributed features:
 'yearbuilt',
 'finishedfloor1squarefeet',
 'taxvaluedollarcnt',
 'regionidzip',
 'latitude'.
4. Taxdelinquencyflag ratio F/T differs significantly. Fireplaceflag - too, but the feature is strongly imbalanced.
5. Property zoning differs significantly for properties and train, but not extremely for train 2016 and 2017.
6. Adversarial feature importances are mostly consistent.
7. Datasets similarity among months: no difference, even none of stat tests showed significance.
8. Datasets similarity among q4 and train: same thing like with months. 
9. Train 2016 vs properties 2016: some distributions are clearly shifted.